# ❄️ Snowflake Arctic Quickstart

In this quickstart you will learn build and evaluate a RAG application with Snowflake Arctic.

Building and evaluating RAG applications with Snowflake Arctic offers developers a unique opportunity to leverage a top-tier, enterprise-focused LLM that is both cost-effective and open-source. Arctic excels in enterprise tasks like SQL generation and coding, providing a robust foundation for developing intelligent applications with significant cost savings. [Learn more about Snowflake Arctic](https://www.snowflake.com/blog/arctic-open-efficient-foundation-language-models-snowflake/)

In this example, we will use Arctic Embed (`snowflake-arctic-embed-m`) as our embedding model via HuggingFace, and Arctic, a 480B hybrid MoE LLM for both generation and as the LLM to power TruLens feedback functions. The Arctic LLM is fully-mananaged by [Cortex LLM functions](https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions)

Note, you'll need to have an active Snowflake account to run Cortex LLM functions from Snowflake's data warehouse.
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/models/arctic_quickstart.ipynb)

In [6]:
# ! pip install trulens_eval chromadb sentence-transformers snowflake-snowpark-python

  Using cached build-1.2.1-py3-none-any.whl.metadata (4.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.4 MB/s eta 0:00:001.6 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tenacity-8.3.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
  Using cached pyproject_hooks-1.1.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached sympy-1.12.1-py3-none-any.whl.metadata (12 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 k

In [8]:
import os
os.environ["SNOWFLAKE_ACCOUNT"] = "xxx-xxxxx" # xxx-xxxx.snowflakecomputing.com
os.environ["SNOWFLAKE_USER"] = "..." 
os.environ["SNOWFLAKE_USER_PASSWORD"] = "..."

In [9]:
from trulens_eval.keys import check_keys
check_keys("SNOWFLAKE_ACCOUNT", "SNOWFLAKE_USER", "SNOWFLAKE_USER_PASSWORD")
from snowflake.snowpark import Session

connection_params = {
    "account": os.environ["SNOWFLAKE_ACCOUNT"],
    "user": os.environ["SNOWFLAKE_USER"],
    "password": os.environ["SNOWFLAKE_USER_PASSWORD"],
}


# Create a Snowflake session
snowflake_session = Session.builder.configs(connection_params).create()

No .env found in /Users/dhuang/Documents/git/trulens/trulens_eval/examples/expositional/models or its parents. You may need to specify secret keys in another manner.


✅ Key SNOWFLAKE_ACCOUNT set from environment.
✅ Key SNOWFLAKE_USER set from environment.
✅ Key SNOWFLAKE_USER_PASSWORD set from environment.


## Get Data

In this case, we'll just initialize some simple text in the notebook.

In [3]:
university_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world.
"""

## Create Vector Store

Create a chromadb vector store in memory.

In [1]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("Snowflake/snowflake-arctic-embed-m")

/opt/anaconda3/envs/trulens/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/opt/anaconda3/envs/trulens/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
document_embeddings = model.encode([university_info])

In [5]:
import chromadb

chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(name="Universities")

Add the university_info to the embedding database.

In [10]:
vector_store.add("uni_info",
                 documents=university_info,
                 embeddings=document_embeddings)

Add of existing embedding ID: uni_info
Insert of existing embedding ID: uni_info


## Build RAG from scratch

Build a custom RAG from scratch, and add TruLens custom instrumentation.

In [11]:
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [105]:
from snowflake.cortex import Complete
import json
class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vector_store.query(
        query_embeddings=model.encode(
        [query], prompt_name="query"),
        n_results=2
    )
        return results['documents']

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        
        def escape_string_for_sql(input_string):
            # Replace backslashes first to avoid double escaping
            escaped_string = input_string.replace('\\', '\\\\')
            # Replace single quotes with double single quotes
            escaped_string = escaped_string.replace("'", "''")
            return escaped_string
        
        prompt=escape_string_for_sql(f"""
         We have provided context information below. 
            {context_str}
            Given this information, please answer the question: {query}
        """)
 
        res = snowflake_session.sql(f"""SELECT SNOWFLAKE.CORTEX.COMPLETE(
            'snowflake-arctic',
            [
            {{'role': 'user', 'content': '{prompt}'}}
            ], {{
                'temperature': 0
            }}
            )""").collect()
       

        completion = json.loads(res[0][0])["choices"][0]["messages"]

        return completion

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion

rag = RAG_from_scratch()

## Set up feedback functions.

Here we'll use groundedness, answer relevance and context relevance to detect hallucination.

In [101]:
from trulens_eval import Feedback, Select
from trulens_eval import Cortex

import numpy as np

# Initialize LiteLLM-based feedback function collection class:
provider = Cortex(model_engine="snowflake-arctic")

# Define a groundedness feedback function
f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

f_coherence = (
    Feedback(provider.coherence_with_cot_reasons, name = "coherence")
    .on_output()
)

ImportError: cannot import name 'Cortex' from 'trulens_eval' (/Users/dhuang/Documents/git/trulens/trulens_eval/trulens_eval/__init__.py)

## Construct the app
Wrap the custom RAG with TruCustomApp, add list of feedbacks for eval

In [106]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'RAG v1',)
    # feedbacks = [f_groundedness, f_answer_relevance, f_context_relevance, f_coherence])

## Run the app
Use `tru_rag` as a context manager for the custom RAG-from-scratch app.

In [107]:
with tru_rag as recording:
    ans = rag.query("Give me a long history of U Dub")

In [108]:
ans

" The University of Washington, also known as U Dub, has a rich history that dates back to its founding in 1861. Established in Seattle, it is a public research university that has grown to accommodate over 45,000 students across its three campuses in Seattle, Tacoma, and Bothell. As the flagship institution of the six public universities in Washington state, UW boasts an impressive infrastructure, encompassing over 500 buildings and 20 million square feet of space.\n\nThe early years of the University of Washington were marked by struggle and perseverance. In its first year, the university opened its doors to just 30 students and a handful of faculty members. Despite these humble beginnings, the institution continued to grow and evolve, eventually becoming one of the leading research universities in the United States.\n\nThroughout the late 19th and early 20th centuries, the University of Washington experienced significant expansion and development. The campus in Seattle underwent mul

In [ ]:
tru.get_leaderboard(app_ids=["RAG v1"])

In [ ]:
tru.run_dashboard()